### Assignment 1 Part III - KG searching, weighting, and embedding

**Overview:** In this notebook, you will learn how to a couple of different ways search to through a large knowledge graph, how to reweight the knowledge graph to favor specific paths, and how to embed nodes in the knowledge graph to try and fill in gaps.   

#### Part I - a couple of different ways search to through a large knowledge graph

These first cells simply set up some variables that will be used below

**IMPORTANT:** You need to obtain the path to your user home folder on the CRC from the shell. You can simply run `pwd` and copy that to replace '/ihome/rboyce/rdb20/' in the cell below.

In [2]:
import os
import sys
USER_HOME='/ihome/rboyce/rdb20/' ## REPLACE THIS WITH THE PATH TO YOUR HOME HOLDER IN THE CRC ENVIRONMENT 
sys.path.append(USER_HOME + '.conda/envs/bionf2071/lib/python3.7/site-packages/')

In [3]:
# There can be many paths of the same length so you can adjust how many you want returned
maxNumPaths = 5

You can populate the list in the next cell with the start and end nodes that you will use to search for paths in the knowledge graph. The [PheKnowLator overview](https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png) shows all of the ontologies loaded into the knowledge graph as orange boxes. You can replace the start and end nodes with entities that interest you in those ontologies by going to [Ontobee](http://www.ontobee.org/) and obtaining the URIs and then replacing the start and end node pairs in the s_o_tpl_L list. Be sure to set FIRST_TO_PROCESS to the first pair. If you have to restart the knowledge graph searches shown later, you can change that variable.

In [4]:
s_o_tpl_L = [
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/DOID_680"),
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/HP_0001297"),
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/HP_0001635"),
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/HP_0002383"),
    ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/HP_0002621")
]
## Leave as None unless you have to restart this cell before it has completely processed all tuples.
## Otherwise, replace this tuple with the first tuple to process completely by the program 
FIRST_TO_PROCESS =  ("http://purl.obolibrary.org/obo/HP_0000716","http://purl.obolibrary.org/obo/DOID_680")

The libraries needed to import the graph and work with it

In [5]:
import os
import os.path
import networkx as nx
import json
import urllib
import traceback
import sys
from itertools import islice
from rdflib import Graph, URIRef, BNode, Namespace, Literal
from rdflib.namespace import RDF, OWL
import smart_open
from node2vec import Node2Vec

The next two cell are the graph loading step. The graph is the output of the PheKnowLator process including filtering out OWL semantics. The sources used in this version of the knowledge graph are listed on [this Wiki page](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources). The graph takes more than 15 gigs of RAM once loaded.

In [ ]:
GRAPHPATH = "PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_Networkx_MultiDiGraph_REWEIGHT_NO_DISJOINT.gpickle"

In [ ]:
# Reloading the graph produced from the PheKnowLator workflow
# The graph 
nx_mdg = nx.read_gpickle(GRAPHPATH)

The next cell defines several functions that will help to explain the results of path searches.

In [ ]:
ontobee_service = "http://sparql.hegroup.org/sparql/"


def query(q,epr,f='application/sparql-results+json'):
    """Function that uses urllib/urllib2 to issue a SPARQL query.
       By default it requests json as data format for the SPARQL resultset"""

    try:
        params = {'query': q}
        params = urllib.parse.urlencode(params)
        opener = urllib.request.build_opener(urllib.request.HTTPHandler)
        request = urllib.request.Request(epr+'?'+params)
        request.add_header('Accept', f)
        request.get_method = lambda: 'GET'
        url = opener.open(request)
        return url.read()
    except Exception as e:
        traceback.print_exc(file=sys.stdout)
        raise e


def pathQuery(pth):
    """Given a single path (list of rdflib objects), run a sparql query to retrieve descriptive information 
       that will help construct a narrative explanation"""
    
    uriL = [x.toPython() for x in filter(lambda x: type(x) == URIRef, pth)]
    subjectStr = ''
    objectStr = ''
    for uri in uriL:
        subjectStr = subjectStr + '?s = <' + uri + '> || '
        objectStr = objectStr + '?o = <' + uri + '> || '
    subjectStr = subjectStr[:-3] # ?s = <..> || ?s = <...>... the URIs for the subject/object entities in the path
    objectStr = objectStr[:-3] # ?o = <..> || ?o = <...>... the URIs for the subject/object entities in the path
   
    q = '''
  prefix obo:<http://purl.obolibrary.org/obo/>
  prefix owl:<http://www.w3.org/2002/07/owl#>
 
  select distinct ?s ?p ?o ?p_lab ?s_lab_eg ?o_lab_eg
  from <http://phenowlator_merged.owl>
  from <ro_with_imports_AD_mods>
  where {{
    ?s ?p ?o.FILTER(({}) && ({})) 
    OPTIONAL{{
     ?p rdfs:label ?p_lab.
   }}
   OPTIONAL{{
     ?egM_s <http://dikb.org/ad#obo_mapping> ?s.
     ?egM_s rdfs:label ?s_lab_eg.
   }}
   OPTIONAL{{
     ?egM_o <http://dikb.org/ad#obo_mapping> ?o.
     ?egM_o rdfs:label ?o_lab_eg.
   }}
  }}
'''.format(subjectStr,objectStr)
    
    return q


def missingLabelQuery(uri, endpoint):
    query_string = '''
SELECT distinct ?lab
WHERE {{ 
 <{}> rdfs:label ?lab. 
}}
'''.format(uri)
    json_string = query(query_string, endpoint)
    resultset = json.loads(json_string)
    rsltsD = {}
    for b in resultset["results"]["bindings"]:
        if not b.get('lab'):
            return None
        
        label = b['lab']['value']
        if label:
            return label
        else:
            return None

def constructPathNarData(pth, sparql_service, debug=False):
    """ Iterate through the path to organize a narrative
        in: pth - a list of rdflib URIRef and BNode objects
        in: sparql_service - URL to the sparql endpoint
    """
    query_string = pathQuery(pth)
    if debug:
        print('[DEBUG] Query:\n' + query_string)
        
    json_string = query(query_string, pheknowlator_service)
    resultset = json.loads(json_string)
    print("[INFO] Number of results: " + str(len(resultset["results"]["bindings"])))
    
    # Re-organize the results set to be a dict keyed by the subject uri
    rsltsD = {}
    for b in resultset["results"]["bindings"]:
        s = b['s']['value']
        if rsltsD.get(s):
            rsltsD[s].append(b)
        else:
            rsltsD[s] = [b]
    
    narrative = ""     
    for i in range(0,len(pth)):
        if i == len(pth) - 1:
            break
        
        n1 = pth[i]
        n2 = pth[i+1]
    
        if not (type(n1) == URIRef and type(n2) == URIRef):
            narrative += "----- BLANK NODE STEP ----"
            continue
        else:
            # locate the results dict that relates n1 to n2 in a subject, predicate, object triple
            o_d = None
            for d in rsltsD.get(n1.toPython()):
                if d['o']['value'] == n2.toPython():
                    o_d = d
                    break
        
            if not o_d:
                print("ERROR: Unable to find a triple relating {} to {} in path {}".format(n1.toPython(),n2.toPython(),pth))
            else:
                o_lab = '<no label found>'
                o_node_id = o_d['o']['value'].split('/')[-1]
                if nodeLabD.get(o_node_id):
                    o_lab = nodeLabD[o_node_id]
                else:
                    if label_cache.get(o_d['o']['value']):
                        o_lab = label_cache[o_d['o']['value']]
                    else:
                        ql = missingLabelQuery(o_d['o']['value'],ontobee_service)
                        if ql:
                            label_cache[o_d['o']['value']] = ql
                            o_lab = ql
                    
                    if o_lab == '<no label found>' and o_d.get('o_lab_eg'):
                        o_lab = o_d['o_lab_eg']['value'] + ' (UMLS label)'

                s_lab = '<no label found>'
                s_node_id = o_d['s']['value'].split('/')[-1]
                if nodeLabD.get(s_node_id):
                    s_lab = nodeLabD[s_node_id]
                else:
                    if label_cache.get(o_d['s']['value']):
                        s_lab = label_cache[o_d['s']['value']]
                    else:
                        ql = missingLabelQuery(o_d['s']['value'],ontobee_service)
                        if ql:
                            label_cache[o_d['s']['value']] = ql
                            s_lab = ql  
                    
                    if s_lab == '<no label found>' and o_d.get('s_lab_eg'):
                        s_lab = o_d['s_lab_eg']['value'] + ' (UMLS label)'

                if o_d.get('p_lab'):                      
                    narrative += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                            s_lab,o_d['p_lab']['value'],o_lab,
                            o_d['s']['value'],o_d['p']['value'],o_d['o']['value']
                           )                          
                else:
                    narrative += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(
                            s_lab,o_d['p']['value'].replace('http://www.w3.org/2000/01/rdf-schema#',''),o_lab,
                            o_d['s']['value'],o_d['p']['value'],o_d['o']['value']
                           )                                          
    return narrative

The next cell loads a file that has all of the node labels which were stripped from the graph when OWL semantics were removed.

In [ ]:
nodeLabD = {}
f = open('PheKnowLator_full_InverseRelations_NotClosed_NoOWLSemantics_NodeLabels.txt','r')
buf = f.read()
f.close()
nodLabL = buf.split('\n')
for line in nodLabL:
    spL = line.split('\t')
    if len(spL) > 1:
        nodeLabD[spL[0]] = spL[1]

This function conducts a shortest path search given a graph source and target k results are returned

In [ ]:
def k_shortest_paths(G, source, target, k, weight='weight'):
    return list(islice(nx.all_shortest_paths(G, source, target, weight=weight), k))

This function conducts the simple path searches given a graph source and target and an integers with the shortest path length known 
The shortest path lengths is used so that already seen simple paths are not returned
k results are returned

In [ ]:
def k_simple_paths(G, source, target, k, shortestLen):
    paths = nx.all_simple_paths(G, source, target, cutoff=shortestLen+20)
    path_l = []
    i = 0
    while i < k:
        try:
            print('[info] applying next operator to search for a simple path of max length {}'.format(shortestLen+20))
            path = next(paths)
        except StopIteration:
            break
        print('[info] Simple path found of length {}'.format(len(path))) 
        if len(path) > shortestLen:
            print('[info] Simple path length greater than shortest path length ({}) so adding to results'.format(shortestLen))
            path_l.append(path)
        i += 1
    return path_l      

In [ ]:
# This is the endpoint where an RDF version of the PheKnowLator graph resides
pheknowlator_service = "https://dbmi-icode-01.dbmi.pitt.edu/sparql"

In [ ]:
# There can be allot of repetition when doing certain search patterns (e.g., confounders) so set up a cache 
processed_tpl_cache = []
# queried label cache
label_cache = {}

The next cell conducts the shortest path searches

In [ ]:
shortestPathsLens = [] # we track the shortest paths from source to target for when we do simple path searches
for tpl in s_o_tpl_L:      
    if FIRST_TO_PROCESS:
        if str(tpl) != str(FIRST_TO_PROCESS):
            print('INFO: skipping tuple because it is not FIRST_TO_PROCESS:' + str(tpl))
            continue
        else:
            FIRST_TO_PROCESS = None
    
    (s,o) = tpl    
    startNd = URIRef(s) 
    endNd = URIRef(o) 
       
    print('INFO: Processing {} and {}:\n'.format(s,o))
                
    try:
        pthL = k_shortest_paths(nx_mdg,startNd,endNd,maxNumPaths)
    except nx.NetworkXNoPath:
        print('INFO: No results in the path search.\n')
        continue
    except nx.NodeNotFound:
        print('INFO: The source node does not exist in the Knowledge Graph.\n')
        continue

    narL = []
    c = -1
    for path in pthL:
        if c == -1:
            shortestPathsLens.append(len(path))
            
        c += 1
        nar = constructPathNarData(path,pheknowlator_service,debug=False)
        print('\n\nINFO: SHORTEST PATH {}:\n'.format(c))
        print(nar)
        print('\n\n')

The next cell conducts the simple path searches but in a way to ignores the shortest paths

In [ ]:
maxNumPaths = 3  # we search for fewer paths b/c this can take much longer than the shortest path searches  
tplCnt = 0
for tpl in s_o_tpl_L:      
    if FIRST_TO_PROCESS:
        if str(tpl) != str(FIRST_TO_PROCESS):
            print('INFO: skipping tuple because it is not FIRST_TO_PROCESS:' + str(tpl))
            continue
        else:
            FIRST_TO_PROCESS = None
    
    (s,o) = tpl    
    startNd = URIRef(s) 
    endNd = URIRef(o) 
       
    print('INFO: Processing {} and {}:\n'.format(s,o))
                
    try:
        pthL = k_simple_paths(nx_mdg, startNd, endNd, maxNumPaths, shortestPathsLens[tplCnt])
        tplCnt += 1
    except nx.NetworkXNoPath:
        print('INFO: No results in the path search.\n')
        continue
    except nx.NodeNotFound:
        print('INFO: The source node does not exist in the Knowledge Graph.\n')
        continue

    narL = []
    c = -1
    for path in pthL:
        c += 1
        nar = constructPathNarData(path,pheknowlator_service,debug=False)
        print('\n\nINFO: SIMPLE PATH {}:\n'.format(c))
        print(nar)
        print('\n\n')

**E7** - Questions about path searches

1. In part II of Assignment I you saw some SPARQL queries over a small knowledge graph created from predications extracted by machine reading with SemMedDB. In this part of the Assignment I you are working with a much larger and more complex graph constructed using ontologies and several large data sources (go back and review sources used in this version of the knowledge graph are listed on [this Wiki page](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources)). We could SPARQL for this graph but we are using graph search instead. Make some obervations in your own words about some differences between the two approaches. 

2. Take a close look at the results of the shortest path searches and make some comments about how potentially factual and useful the path narratives would be for providing a mechanistic explanation of the relationship between the start and end nodes. 

3. Similar to the previous question, take a close look at the results of the SIMPLE path searches and make some comments about how potentially factual and useful the path narratives would be for providing a mechanistic explanation of the relationship between the start and end nodes. Also, what differences between the simple and shortes path searches stand out to you.

4. Think back to the previous part of Assignment I where we saw transitive and symmetric closure over a small knowledge graph created from SemMedDB predications. The PheKnoLator knowledge graph we are working with here is built as a formal ontology represented in OWL. Can you suggest how to approach closure for this large knowledge graph (hint: think about what can be done with description logic)? Also, explain how you think closure would be important for the path search approach you see in this notebook. 
 
6. Do you have any questions for me about this section?

#### Part II - how to reweight the knowledge graph to favor specific paths

There are a number of graph metrics that could be use to reweight edges in the graph (see https://networkx.org/documentation/stable/reference/algorithms/centrality.html) Here, we pick a simple measure to calculate called degree centrality.

In [ ]:
## obtain a dictionary of node degree centrality for all nodes using the default parameters
centrality = nx.degree_centrality(nx_mdg)

In [ ]:
# Using Alzheimer's disease as an example 
centrality[URIRef('http://purl.obolibrary.org/obo/HP_0002511')]

In [ ]:
# Using Cell as an example
centrality[URIRef('http://purl.obolibrary.org/obo/CL_0000000')]

In [ ]:
# We are going to use node degree centrality when reweighting the graph so we need to calculate the measure for all nodes
f = open('centrality.tsv','w')
f.write('node\tdegree_centrality\n')
for k,v in centrality.items():
    f.write('{}\t{}\n'.format(k.toPython(),v))
f.close()

The next cell creates a new graph  copying and reweighting the current graph using degree centrality

In [ ]:
nx_mdg_cntr = nx.MultiDiGraph()

# the reweight graph will remove 'disjoint' and 'domain' predicates  and apply a 
edge_key = -1 # an int that we will increment to uniquely identify edges
for s, o, data in nx_mdg.edges(data=True):
    p = data['predicate']
    
    edge_key += 1
    
    ## default weight = 2
    weight = 2.0
    
    # Skip disjoint and domain predicates
    if p.toPython() == 'http://www.w3.org/2002/07/owl#disjointWith' or p.toPython() == 'http://www.w3.org/2000/01/rdf-schema#domain':
        continue    
           
    # Add to the weight the degree centrality of each node
    if centrality.get(p):
        weight = weight + centrality[p]
    if centrality.get(o):
        weight = weight + centrality[o]
    
    # add the edge to the graph giving it a unique key and weight
    nx_mdg_cntr.add_edge(s, o, **{'predicate': p,'key': edge_key, 'weight':weight})

In [ ]:
nx.write_gpickle(nx_mdg_cntr,'PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_DEGREE_CNTRLTY_REWEIGHT_NO_DISJOINT.gpickle')

Conduct shortest and simple path searches with the reweighted graph

In [ ]:
maxNumPaths = 3 
shortestPathsLens = [] # we track the shortest paths from source to target for when we do simple path searches
for tpl in s_o_tpl_L:      
    if FIRST_TO_PROCESS:
        if str(tpl) != str(FIRST_TO_PROCESS):
            print('INFO: skipping tuple because it is not FIRST_TO_PROCESS:' + str(tpl))
            continue
        else:
            FIRST_TO_PROCESS = None
    
    (s,o) = tpl    
    startNd = URIRef(s) 
    endNd = URIRef(o) 
       
    print('INFO: Processing {} and {}:\n'.format(s,o))
                
    try:
        pthL = k_shortest_paths(nx_mdg_cntr,startNd,endNd,maxNumPaths)
    except nx.NetworkXNoPath:
        print('INFO: No results in the path search.\n')
        continue
    except nx.NodeNotFound:
        print('INFO: The source node does not exist in the Knowledge Graph.\n')
        continue

    narL = []
    c = -1
    for path in pthL:
        if c == -1:
            shortestPathsLens.append(len(path))
            
        c += 1
        nar = constructPathNarData(path,pheknowlator_service,debug=False)
        print('\n\nINFO: SHORTEST PATH {}:\n'.format(c))
        print(nar)
        print('\n\n')

In [ ]:
# Conduct SIMPLE path searches with the reweighted graph
maxNumPaths = 3  
tplCnt = 0
for tpl in s_o_tpl_L:      
    if FIRST_TO_PROCESS:
        if str(tpl) != str(FIRST_TO_PROCESS):
            print('INFO: skipping tuple because it is not FIRST_TO_PROCESS:' + str(tpl))
            continue
        else:
            FIRST_TO_PROCESS = None
    
    (s,o) = tpl    
    startNd = URIRef(s) 
    endNd = URIRef(o) 
       
    print('INFO: Processing {} and {}:\n'.format(s,o))
                
    try:
        pthL = k_simple_paths(nx_mdg_cntr, startNd, endNd, maxNumPaths, shortestPathsLens[tplCnt])
        tplCnt += 1
    except nx.NetworkXNoPath:
        print('INFO: No results in the path search.\n')
        continue
    except nx.NodeNotFound:
        print('INFO: The source node does not exist in the Knowledge Graph.\n')
        continue

    narL = []
    c = -1
    for path in pthL:
        c += 1
        nar = constructPathNarData(path,pheknowlator_service,debug=False)
        print('\n\nINFO: SIMPLE PATH {}:\n'.format(c))
        print(nar)
        print('\n\n')

**E8** - questions about graph editing and reweighting

1. Think about how degree centrality is being used here to reweight edges in the graph. If the goal is to obtain mechanistically accurate narratives, does using this measure to reweight make sense?

2. Imagine that the goal is to avoid the inclusion several 'hub' nodes in path results. Look over the [centrality measures](https://networkx.org/documentation/stable/reference/algorithms/centrality.html) and justify if another which measure(s) might be better than degree centraity to accomplish this task.

3. How could you use knowledge about the Relation Ontology to bias path search results to favor certain kinds of nodes (e.g., pathways) over others (e.g., genes) (hint: look at the ontologies and RO entities [here](https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png))?

4. Pick three different centrality measures and explain the computational complexity tradeoffs of each.



### Part III - how to embed nodes in the knowledge graph to try and fill in gaps

We're going to tackle link prediction as a supervised learning problem on top of node representations/embeddings. The embeddings are computed with the unsupervised node2vec algorithm. After obtaining embeddings, a binary classifier can be used to predict a link, or not, between any two nodes in the graph. Various hyperparameters could be relevant in obtaining the best link classifier -

There are four steps:

1. Obtain embeddings for each node
2. For each set of hyperparameters, train a classifier
3. Select the classifier that performs the best
4. Evaluate the selected classifier on unseen data to validate its ability to generalise

This part of the notebook has been adapted from a demo of the StellarGraph library (https://stellargraph.readthedocs.io/en/stable/index.html) for link prediction using the node2vec algorithm.

<a name="refs"></a>
**References:** 

[1] Node2Vec: Scalable Feature Learning for Networks. A. Grover, J. Leskovec. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2016.

First, we derive a simple undirected graph that has only predicates (edges) indicitive of interaction:

http://purl.obolibrary.org/obo/RO_0002434 - interacts with

http://purl.obolibrary.org/obo/RO_0002436 - molecularly interacts with

http://purl.obolibrary.org/obo/RO_0002435 - geneticly interacts with

http://purl.obolibrary.org/obo/RO_0002437 - biotically interacts with

http://purl.obolibrary.org/obo/RO_0000056 - partipates in

Note: We limit the graph size to be a few tens of thousands of nodes (the full graph is >200K nodes) so that embedding can happen in a relatively short amount of time 

We also need to be choosy about the nodes we select because there are some more general nodes that have many thousands of interaction relationships.

In [ ]:
nx_mdg_interacts = nx.Graph()

object_nodeD = {} # tracking object nodes so we can add only those that are the object of 2 or more interactions
maxNodes = 6000 # the max count of unique subject nodes
edge_key = -1 # an int that we will increment to uniquely identify edges
s_node_visitedD = {} # tracks unique subject nodes 
s_count = 0 # some entitites have a huge number of interactions so we will only sample a max of 100 for any entity
s_current = None
for s, o, data in nx_mdg.edges(data=True):
    if not s_node_visitedD.get(s):
        s_node_visitedD[s] = 1
        if len(s_node_visitedD) == maxNodes:
            break
    
    if s == s_current:
        s_count += 1
        if s_count > 50:
            continue
    else:
        s_current = s
        s_count = 0 
        
    p = data['predicate']
    
    edge_key += 1
    
    if not (p.toPython() == 'http://purl.obolibrary.org/obo/RO_0002434' # interacts with
            or p.toPython() == 'http://purl.obolibrary.org/obo/RO_0002436' # molecularly interacts with
            or p.toPython() == 'http://purl.obolibrary.org/obo/RO_0002435' # genetically interacts with
            or p.toPython() == 'http://purl.obolibrary.org/obo/RO_0002437' # biotically interacts with 
            or p.toPython() == 'http://purl.obolibrary.org/obo/RO_0000056' # partipates in
           ):
        continue    
    
    if object_nodeD.get(o):
        object_nodeD[o] += 1
        
        # add the edge to the graph giving it a unique key 
        nx_mdg_interacts.add_edge(s, o, **{'predicate': p,'key': edge_key})
        
    else:
        object_nodeD[o] = 1

In [ ]:
nx.write_gpickle(nx_mdg_interacts,'PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_INTERACTS.gpickle')

In [ ]:
## Uncomment this to reload the 'interacts only' graph from file if needed
# nx_mdg_interacts = nx.read_gpickle('PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_INTERACTS.gpickle')

The next few cells give some insight into the content of our interaction sub-graph

In [ ]:
print(nx.info(nx_mdg_interacts))

In [ ]:
i = 0 
for d in nx_mdg_interacts.degree():
    if d[1] > 2:        
        print(d)
        i += 1
        if i == 100:
            break

In [ ]:
# A sample of what this new graph looks like
i = 0
for s, o, data in nx_mdg_interacts.edges(data=True):
    if i == 200:
        break
    
    print('{}\t{}\t{}'.format(s,o,data))
    i += 1

#### Node2Vec

We use Node2Vec [[1]](#refs), to calculate node embeddings. These embeddings are learned in such a way to ensure that nodes that are close in the graph remain close in the embedding space. Node2Vec first involves running random walks on the graph to obtain our context pairs, and using these to train a Word2Vec model.

These are the set of parameters we can use:

* `p` - Random walk parameter "p"
* `q` - Random walk parameter "q"
* `dimensions` - Dimensionality of node2vec embeddings
* `num_walks` - Number of walks from each node
* `walk_length` - Length of each random walk
* `window_size` - Context window size for Word2Vec
* `num_iter` - number of SGD iterations (epochs)
* `workers` - Number of workers for Word2Vec

There are a few steps involved in using the model to perform link prediction:
1. We calculate link/edge embeddings for the positive and negative edge samples by applying a binary operator on the embeddings of the source and target nodes of each sampled edge.
2. Given the embeddings of the positive and negative examples, we train a logistic regression classifier to predict the probability indicating whether an edge between two nodes should exist or not.
3. To use the embeddings in a traditional machine learning classifier, we use binary operators on the embeddings such as average, Hadamard, L1, L2. Here we use the 'average operator' with node embeddings in the logistic regression classifier.


The next cells aply the node2vec to conctruc a 30 dimension embedding of the interaction sub-graph using 5 random walks per node with up to 10 node steps each. These hyperparameters are not tuned.

In [ ]:
node2vec = Node2Vec(nx_mdg_interacts, dimensions=30, walk_length=10, num_walks=5, workers=1)

In [ ]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [ ]:
model.save('PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_INTERACTS.model')

In [ ]:
## Uncomment this and run it to reload the model from a file if you have already created it
from gensim.models import Word2Vec
model = Word2Vec.load('PheKnowLator_full_InverseRelations_NotClosed_OWLNETS_INTERACTS.model')

We can examine the embedding vectors for any node in the graph.

In [ ]:
# entinostat
model.wv.get_vector('http://purl.obolibrary.org/obo/CHEBI_132082')

In [ ]:
model.wv.most_similar('http://purl.obolibrary.org/obo/CHEBI_132082')

In [ ]:
# positive regulation of nervous system development
model.wv.get_vector('http://purl.obolibrary.org/obo/GO_0051962')

We can also seek the most similar nodes to a given ndode in the embedding space 

In [ ]:
model.wv.most_similar('http://purl.obolibrary.org/obo/GO_0051962')

In [ ]:
# positive=purine ribonucleoside binding and negative=positive regulation of nervous system development
model.wv.most_similar(positive=['http://purl.obolibrary.org/obo/GO_0032550'],negative=['http://purl.obolibrary.org/obo/GO_0051962'])

The next cells train a model for link prediction using the embeddings. We start by building an adjacency matrix and then traversing it to find the positions of the zeros which represent node pairs that are not connected in the graph.  

The steps shown here follow the same procedure used in this [nice tutorial by Prateek Joshi](https://www.analyticsvidhya.com/blog/2020/01/link-prediction-how-to-predict-your-future-connections-on-facebook/) but applied to the interaction sub-graph of the PheKnowLator graph.  

In [ ]:
nodelist = []
nds = nx_mdg_interacts.nodes()
initial_node_count = nx.number_of_nodes(nx_mdg_interacts)
for n in nds:
    nodelist.append(n)

In [ ]:
# build adjacency matrix
adj_G = nx.to_numpy_matrix(nx_mdg_interacts,nodelist)

In [ ]:
adj_G.shape

In [ ]:
# get unconnected node-pairs
all_unconnected_pairs = [None]*(adj_G.shape[0]*adj_G.shape[1])
all_connected_pairs = [None]*(adj_G.shape[0]*adj_G.shape[1])

# traverse adjacency matrix (iterate columns by rows)
offset = 0
for i in range(adj_G.shape[0]):
  for j in range(offset,adj_G.shape[1]):
    if i != j:
        if adj_G[i,j] == 0:
          all_unconnected_pairs[i+j] = (nodelist[i],nodelist[j])
        else:            
          all_connected_pairs[i+j] = (nodelist[i],nodelist[j])

  offset = offset + 1

In [ ]:
import pandas as pd

all_connected_pairs_clean = [x for x in filter(lambda x: x != None, all_connected_pairs)]
node_1_linked = [i[0] for i in all_connected_pairs_clean]
node_2_linked = [i[1] for i in all_connected_pairs_clean]
original_g_df = pd.DataFrame({'node_1':node_1_linked, 
                              'node_2':node_2_linked})

all_unconnected_pairs_clean = [x for x in filter(lambda x: x != None, all_unconnected_pairs)]
node_1_unlinked = [i[0] for i in all_unconnected_pairs_clean]
node_2_unlinked = [i[1] for i in all_unconnected_pairs_clean]
data = pd.DataFrame({'node_1':node_1_unlinked, 
                     'node_2':node_2_unlinked})

# add target variable 'link'
data['link'] = 0

In [ ]:
import pickle
f = open('original_g_df.pickle','wb')
pickle.dump(original_g_df,f)
f.close()

f = open('data.pickle','wb')
pickle.dump(data,f)
f.close()

In [ ]:
## Use this cell to reload if needed
#import pickle
#f = open('original_g_df.pickle','rb')
#original_g_df = pickle.load(f)
#f.close()

#f = open('data.pickle','rb')
#data = pickle.load(f)
#f.close()

In [ ]:
original_g_df_temp = original_g_df.copy()

# empty list to store removable links
omissible_links_index = []

ctr = 0
ctr2 = 0
for i in original_g_df.index.values:
    # remove a node pair and build a new graph
    G_temp = nx.from_pandas_edgelist(original_g_df_temp.drop(i), "node_1", "node_2", create_using=nx.Graph())
    
    #print('len(G_temp.nodes): {}'.format(nx.number_of_nodes(G_temp)))
    
    ctr2 += 1
    if ctr2 % 1000 == 0:
        print(str(ctr2))
    
    # check that the number of nodes is same as the resulting graph
    if (nx.number_of_nodes(G_temp) == initial_node_count):                                                            
        omissible_links_index.append(i)
        original_g_df_temp = original_g_df_temp.drop(index = i)
        ctr += 1
        if ctr % 500 == 0:
            print('[info] Count of admissable links: {}'.format(ctr))
        if ctr == 2000:
            break

In [ ]:
len(omissible_links_index)

In [ ]:
f = open('omissible_links_index.pickle','wb')
pickle.dump(omissible_links_index,f)
f.close()

In [ ]:
# create dataframe of removable edges
original_g_df_ghost = original_g_df.loc[omissible_links_index]

# add the target variable 'link'
original_g_df_ghost['link'] = 1

# Reduce the dataset containing non-connected nodes to 4K so that there is a 1:2 ratio of connected nodes to non-connected nodes
# The reduced dataset is a random sample without replacement
data_reduced = data.sample(4000)

data_reduced = data_reduced.append(original_g_df_ghost[['node_1', 'node_2', 'link']], ignore_index=True)

In [ ]:
model.wv.get_vector('http://purl.obolibrary.org/obo/CHEBI_2430')

In [ ]:
x = [model.wv.get_vector(i.toPython()) + model.wv.get_vector(j.toPython()) for i,j in zip(data_reduced['node_1'], data_reduced['node_2'])]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import numpy as np
xtrain, xtest, ytrain, ytest = train_test_split(np.array(x), 
                                                data_reduced['link'], 
                                                test_size = 0.3, 
                                                random_state = 35)

Two baseline classifiers with which to compare the LR using embedding features

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(xtrain, ytrain)
predictions = dummy_clf.predict_proba(xtest)
roc_auc_score(ytest, predictions[:,1])

In [ ]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(xtrain, ytrain)
predictions = dummy_clf.predict_proba(xtest)
roc_auc_score(ytest, predictions[:,1])

In [ ]:
yhat = dummy_clf.predict(xtest)
d_f1 = f1_score(ytest, yhat)
print('F1: {}'.format(d_f1))

LR using the node2vec embeddings

In [ ]:
lr = LogisticRegression(class_weight="balanced")
lr.fit(xtrain, ytrain)

In [ ]:
predictions = lr.predict_proba(xtest)

In [ ]:
roc_auc_score(ytest, predictions[:,1])

In [ ]:
yhat = lr.predict(xtest)
lr_f1 = f1_score(ytest, yhat)
print('F1: {}'.format(lr_f1))

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

lr_precision, lr_recall, _ = precision_recall_curve(ytest, predictions[:,1])

plt.plot(lr_recall, lr_precision, marker='.', label='Logistic')

# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
plt.show()


**E9** - questions about graph embedding and link prediction

1. List 3 applications of network embeddings in biomedicine. For each embedding application, name an ontology that you believe would be useful in constructing the knowledge graph that would be used to create the embeddings.

2. Node embedding methods (such as node2vec and DeepWalk) use node features to embed graph information into vector space. Explain ‘node features’ with a few examples.

3. List a few data sources used to create the kg-covid-19 knowledge graph and find examples of triples that are included in the graph. 

4. Current graph representation learning (GRL) methods are not able to embed temporal graphs. Find an application in biomedicine where temporal graphs would be useful for representing knowledge.

5. In this vignette, did we do link prediction on a homogeneous or heterogenious graph? 

6. Given its performance, would you trust for predicting novel links that are currently not in the graph? Please explain your answer. 

7. We did not tune any hyperparameters for the embedding. What options would you be interested in trying out?

8. Some researchers have experimented with embedding knowledge graph nodes in hyperbolic geometry. Do some searching about this topic and write a sentence or two on what you find to be the main motivation of researchers for trying this out. Include citations.

9. Write any comments or questions that you have about the embedding excercise in this notebook.

**ALL DONE**